In [65]:
############################################################################################################
#                                  Author: Flora Medeiros Sauerbronn                                       #
#                                           Date: 02/01/2024                                               #
#      This routine creates and cleans csv files from the selections that Andrea Dalben created            # 
#                          This files have whisles and click of dolphins                                   #
############################################################################################################

In [66]:
import csv
import pandas as pd
import numpy as np
import os 

In [67]:
#COMEÇANDO COM OS ARQUIVOS DE VALIDAÇÃO
lab_click = pd.read_csv('D:/LABELS_ANDREA_AUDIOS/annotation_val_clicks.csv')

#listando o nome dos audios que estão no arquivo
audio_names = lab_click['filename'].drop_duplicates()
#transformando em lista pra facilitar
audio_names = audio_names.tolist()

In [68]:
#Agoora vamos fazer um loop que vai procurar cada arquivo desses e vai juntar todos em um
#Ou seja, ele vai separar os arquivos de train, test and val
# Caminho para o arquivo CSV vazio 
caminho_csv_vazio = 'D:/LABELS_ANDREA_AUDIOS/selections/annotation_val_all_FS.csv' #PRECISA ALTERAR QUAL ARQUIVO VOCÊ ESTA FAZENDO (test,train or val ?)
# Cria um DataFrame vazio
df_final = pd.DataFrame()




for i in range(len(audio_names)):
    file =f'D:/LABELS_ANDREA_AUDIOS/selections/selections/{audio_names[i]}.Table.1.selections.txt'

    #Reading the file
    #df_atual = pd.read_csv(file)
    # Lendo o arquivo
    df_atual = pd.read_csv(file, sep='\t')  # Adiciona sep='\t' para usar tabulação como delimitador
    df_atual.columns = df_atual.columns.str.capitalize()
    df_atual['filename'] = audio_names[i]
     # Calculando a coluna 'call_length'
    df_atual['call_length'] = df_atual['End time (s)'] - df_atual['Begin time (s)']

     # Substituindo valores na coluna 'label'
     # Corrigindo os erros ortograficos
    df_atual['Type'] = df_atual['Type'].str.capitalize()
    df_atual['Type'] = df_atual['Type'].replace({'^W.*': 'W', '^C.*': 'C'}, regex=True)
    df_atual['Type'] = df_atual['Type'].replace({'W': 2, 'C': 1})

    #Adding the file
    df_final = pd.concat([df_final, df_atual], ignore_index=True)



In [69]:
# Selecionando apenas as colunas desejadas
df_final = df_final[['Channel', 'Begin time (s)', 'End time (s)', 'filename','call_length','Type']]



    # Renomeando a coluna 'Type' para 'new_type' (substitua 'new_type' pelo nome desejado)
df_final = df_final.rename(columns={'Channel':'channel','Type': 'label','Begin time (s)': 'begin_time','End time (s)':'end_time'})

#VERIFICA EXPURIOS
#########################################
d1 = df_final['label'].drop_duplicates()
d2 = df_final['label'].value_counts()
#########################################


valores_desejados = [1, 2]

# Filtra as linhas onde a coluna 'label' contém os valores desejados ou NaN
# Mantém os valores desejados
df_final = df_final[df_final['label'].isin(valores_desejados)]

# Remove os NaN
df_final = df_final.dropna(subset=['label'])
df_final['label'] = df_final['label'].astype(int)
# Saving the file
df_final.to_csv(caminho_csv_vazio, index=False, sep=',')

In [70]:
#Agora Precisamos criar o label 0
#Este label significa que é o ruído, ou seja, os egativos, quando não há sinal. 
#Isso será todas as vezes que não tem sinal

In [71]:
df_final
#Ordenar por Arquivo, Canal, Begin_Time
#Precisa saber qual o tamanho total do arquivo para fazer recortes de ruido.
#Que ai vemos os gaps do início ao fim que não estão com labels e eles todos viram ruídos
#com o label de zero (0)

,channel,begin_time,end_time,filename,call_length,label
0,3,169.106850,175.931075,PAM_MF_20190204_064000_000,6.824225,1
1,2,230.594003,233.915764,PAM_MF_20190204_064000_000,3.321760,1
2,3,282.068016,283.752812,PAM_MF_20190204_064000_000,1.684797,1
3,3,330.119295,331.766888,PAM_MF_20190204_064000_000,1.647593,1
4,4,330.353147,331.713740,PAM_MF_20190204_064000_000,1.360593,1
...,...,...,...,...,...,...
239,2,13.593399,16.916570,PAM_MF_20190130_220942_542,3.323171,1
240,4,14.801362,16.335525,PAM_MF_20190130_220942_542,1.534163,1
241,4,16.998121,17.457990,PAM_MF_20190130_220942_542,0.459869,1
242,3,13.180552,16.304944,PAM_MF_20190130_220942_542,3.124392,1


In [72]:
#Ordenando o dataframe
df_ordenado = df_final.sort_values(by=['filename', 'channel', 'begin_time'])
df_ordenado

,channel,begin_time,end_time,filename,call_length,label
53,2,9.556206,9.987987,LPS1142017_MF_20180723_085055_579,0.431781,1
83,2,92.656542,93.152841,LPS1142017_MF_20180723_085055_579,0.496300,1
109,2,372.031991,372.389327,LPS1142017_MF_20180723_085055_579,0.357336,1
112,2,406.274151,406.668709,LPS1142017_MF_20180723_085055_579,0.394558,1
108,3,363.218959,363.650740,LPS1142017_MF_20180723_085055_579,0.431781,1
...,...,...,...,...,...,...
9,4,146.499868,148.312622,PAM_MF_20190205_040000_000,1.812754,1
12,4,151.791302,152.333434,PAM_MF_20190205_040000_000,0.542132,1
15,4,161.961924,163.051835,PAM_MF_20190205_040000_000,1.089911,1
20,4,182.184576,183.969093,PAM_MF_20190205_040000_000,1.784518,1


In [73]:
#Função que descobre a duração em segundos de um audio
import wave

def obter_duracao_wav(caminho_arquivo):
    with wave.open(caminho_arquivo, 'rb') as arquivo_wav:
        # Obter a taxa de amostragem (número de frames por segundo)
        taxa_amostragem = arquivo_wav.getframerate()

        # Obter o número total de frames no arquivo
        total_frames = arquivo_wav.getnframes()

        # Calcular a duração em segundos
        duracao_segundos = total_frames / taxa_amostragem

    return duracao_segundos


In [74]:

# Exemplo de uso
caminho_do_arquivo_wav = 'D:/AUDIOS/test/PAM_MF_20190101_021441_034.wav'

duracao = obter_duracao_wav(caminho_do_arquivo_wav)
print(f'Duração do arquivo WAV: {duracao:.2f} segundos')

Duração do arquivo WAV: 318.97 segundos


In [75]:
#O que fazemos é. Pegamos o primeiro sinal de um audio. 
#1 - Desse begin time pra trás é ruido. E ai fazemos isso pra todos os canais. 
#2 -  O segundo sinal de um audio - Desse bein_time do segundo sinal, até o end_time do primeiro sinal é ruido.
#3 - O último sinal que esse audio tiver, pegamos o seu end_time e vamos até (duracao - 0.001) e fazemos pra todos os canais
#Precisa de um contador para os sinais 

In [76]:
#Olha linha por linha os dados ordenados em filaname, channel e begintime
#Se begin time for <> 0:
        #Tudo que vem antes é ruido
        #Em seguida tudo do end_time[i] até o begin_time[i+1] é ruido
        #vai fazer isso até

In [77]:
df = pd.DataFrame(df_ordenado)

In [78]:
def adicionar_linhas_ruido(df):
    linhas_ruido = []
    for _, grupo in df.groupby(['filename', 'channel']):
        grupo = grupo.reset_index(drop=True)
        primeiro_sinal = grupo.iloc[0]
        ultimo_sinal = grupo.iloc[-1]

        # Adicionar linha de ruído antes do primeiro sinal
        linha_ruido_inicio = {'channel': primeiro_sinal['channel'],
                              'begin_time': 0,
                              'end_time': primeiro_sinal['begin_time'],
                              'filename': primeiro_sinal['filename'],
                              'call_length': 0,
                              'label': 0}
        linhas_ruido.append(linha_ruido_inicio)

        # Adicionar linhas de ruído entre os sinais
        for i in range(len(grupo) - 1):
            linha_ruido_entre = {'channel': grupo.at[i, 'channel'],
                                 'begin_time': grupo.at[i, 'end_time'],
                                 'end_time': grupo.at[i + 1, 'begin_time'],
                                 'filename': grupo.at[i, 'filename'],
                                 'call_length': 0,
                                 'label': 0}
            linhas_ruido.append(linha_ruido_entre)

        # Adicionar linha de ruído após o último sinal
        linha_ruido_fim = {'channel': ultimo_sinal['channel'],
                           'begin_time': ultimo_sinal['end_time'],
                           'end_time': ultimo_sinal['end_time'] + 0.001,
                           'filename': ultimo_sinal['filename'],
                           'call_length': 0,
                           'label': 0}
        linhas_ruido.append(linha_ruido_fim)

    novo_df = pd.DataFrame(linhas_ruido, columns=df.columns)
    return novo_df


In [79]:

# Adicionar linhas de ruído ao DataFrame original
df_com_ruido = pd.concat([df_ordenado, adicionar_linhas_ruido(df_ordenado)])

# Ordenar o DataFrame resultante
df_com_ruido = df_com_ruido.sort_values(by=['filename', 'channel', 'begin_time'])

# Resetar índices
df_com_ruido = df_com_ruido.reset_index(drop=True)

# Exibir o DataFrame resultante
print(df_com_ruido)


     channel  begin_time    end_time                           filename  \
0          2    0.000000    9.556206  LPS1142017_MF_20180723_085055_579   
1          2    9.556206    9.987987  LPS1142017_MF_20180723_085055_579   
2          2    9.987987   92.656542  LPS1142017_MF_20180723_085055_579   
3          2   92.656542   93.152841  LPS1142017_MF_20180723_085055_579   
4          2   93.152841  372.031991  LPS1142017_MF_20180723_085055_579   
..       ...         ...         ...                                ...   
506        4  163.051835  182.184576         PAM_MF_20190205_040000_000   
507        4  182.184576  183.969093         PAM_MF_20190205_040000_000   
508        4  183.969093  189.853483         PAM_MF_20190205_040000_000   
509        4  189.853483  190.677976         PAM_MF_20190205_040000_000   
510        4  190.677976  190.678976         PAM_MF_20190205_040000_000   

     call_length  label  
0       0.000000      0  
1       0.431781      1  
2       0.000000     

In [80]:
df_com_ruido.to_csv(caminho_csv_vazio, index=False, sep=',')